# Storyline
This notebook organizes all the results in the "analysis" folder within a concrete storyline

In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

In [2]:
%load_ext autoreload
%autoreload 1
%aimport utils_read_parsing
from utils_timeline_viz import *
from utils_read_parsing import *
import matplotlib.pyplot as plt
from tabulate import tabulate
from scipy.stats import mannwhitneyu
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')
matplotlib.rcParams['figure.figsize'] = 7, 4
pd.set_option('precision',3)
np.set_printoptions(precision=3,suppress=True)

In [4]:
# pre_df = pd.read_csv('pre_survey_results.txt', sep='\t')
table_cvs_df = pd.read_csv('table_cvs_results.txt', sep='\t')
graph_cvs_df = pd.read_csv('graph_cvs_results.txt', sep='\t')

In [ ]:
metadf = get_student_metadata()
order = dict(zip(metadf.index,metadf['activity order']))
graph_cvs_df['activity order'] = graph_cvs_df.studentid.apply(lambda sid: order[sid])
table_cvs_df['activity order'] = table_cvs_df.studentid.apply(lambda sid: order[sid])

In [ ]:

variables = ["Battery voltage","Area","Separation","Wavelength","Width","Concentration"]

def binarize(number):
    if number>0 : return 1
    else: return 0

graph_cvs_df2 = graph_cvs_df.copy()
for v in variables:
    graph_cvs_df2[v] = graph_cvs_df.apply(lambda row: binarize(row[v]), axis=1)

table_cvs_df2 = table_cvs_df.copy()
for v in variables:
    table_cvs_df2[v] = table_cvs_df.apply(lambda row: binarize(row[v]), axis=1)

graph_cvs_df2['sum'] = graph_cvs_df2[variables].sum(axis=1)
table_cvs_df2['sum'] = table_cvs_df2[variables].sum(axis=1)
table_cvs_df2['sum caps'] = table_cvs_df2[['Battery voltage','Area','Separation']].sum(axis=1)
table_cvs_df2['sum light absorbance'] = table_cvs_df2[["Wavelength","Width","Concentration"]].sum(axis=1)
graph_cvs_df2['sum caps'] = graph_cvs_df2[['Battery voltage','Area','Separation']].sum(axis=1)
graph_cvs_df2['sum light absorbance'] = graph_cvs_df2[["Wavelength","Width","Concentration"]].sum(axis=1)
table_cvs_df2.head()

# Preliminary results

## Student simulation experience
Are students used to using simulations and have they covered either topic before?

## Learning in the PhET
Let's look at pre to post to see if students learn by condition (order).

Post activity model of sim doesn’t depend on incoming knowledge. This results suggests that the use of inquiry strategies is what determines leaning/model quality

# How can inquiry strategies be detected through log data in a PhET sim activity?

## Detect two behaviors, or potential strategies: collection of CVS in table and the plotting of CVS data in a graph
How often is CVS used per variable?

Quantitative variables are similar, so let's concentrate on those.

## Motivation for splitting into two behaviors
Let's compare the two strategies in terms of sampling

## Relation between behavior and pre score

Use of strategy doesn’t depend on pre

## Relation between behavior and pre score

Yes use of strategies changes outcome! Suggesting that the captured behaviors are deliberate inquiry strategies employed by students

# How does using strategy help student learn?

## Sequential or not

## Number of points used

## Consistency by sim 

# How does the use of inquiry strategies change over time? Do students change their inquiry strategies over time and if so for the better of their post scores?

## Consistency over time: proficient users of strategy are consistent, less proficient are less consistent

## Order increases use of strategy and order increases post score

## Use of CVS in table vs. graph over time